# Assignment 4, Aayush Verma,30/07/2024

## Question 1

### Transformer modules are defined in associated Python File

In [1]:
from pyTorchTransformer import Transformer
from transformers import AutoConfig,AutoTokenizer
import torch
from datasets import load_dataset
from tqdm.notebook import tqdm
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
srctokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")
tgttokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

In [3]:
trf = Transformer().cuda()

In [4]:
src = srctokenizer(["Hello World!"],padding = True, truncation = True, max_length = 256,return_tensors = "pt")
tgt = tgttokenizer(["Hello World!"],padding = True, truncation = True, max_length = 256,return_tensors = "pt")

In [5]:
trf(src.input_ids.cuda(),tgt.input_ids[:,:-1].cuda()).shape

torch.Size([1, 4, 105879])

In [6]:
ds = load_dataset("Aarif1430/english-to-hindi")

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['english_sentence', 'hindi_sentence'],
        num_rows: 127705
    })
})

In [8]:
ds = ds['train'].train_test_split(test_size = 0.1)

In [9]:
train = ds['train']
test = ds['test']

In [10]:
train = train.train_test_split(test_size = 0.005)

In [11]:
val = train['test']
train = train['train']

In [12]:
val

Dataset({
    features: ['english_sentence', 'hindi_sentence'],
    num_rows: 575
})

In [13]:
train

Dataset({
    features: ['english_sentence', 'hindi_sentence'],
    num_rows: 114359
})

In [14]:
train = torch.utils.data.DataLoader(train,batch_size = 8,num_workers = 1)
val = torch.utils.data.DataLoader(val,batch_size = 8,num_workers = 1)

In [15]:
optimizer = torch.optim.AdamW(trf.parameters(),lr = 3e-5)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1.0, total_iters=10000)
CCE = torch.nn.CrossEntropyLoss(ignore_index = 0)

In [16]:
step = 0
total_loss = 0.0
best_val_loss = 10000
for epoch in range(10):
    for batch in tqdm(train):
        trf.train()
        src = srctokenizer(batch['english_sentence'],padding = True, truncation = True, max_length = 256,return_tensors = "pt").input_ids.cuda()
        tgt = tgttokenizer(batch['hindi_sentence'],padding = True, truncation = True, max_length = 256,return_tensors = "pt").input_ids.cuda()
        optimizer.zero_grad()
        logits = trf(src,tgt[:,:-1]).permute((0,2,1))
        loss = CCE(logits,tgt[:,1:])
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()*src.size(0)
        step+= 1
        if step%1000 == 0:
            val_loss = 0.0
            accuracy = 0.0
            for valBatch in tqdm(val,leave=False):
                with torch.no_grad():
                    trf.eval()
                    src = srctokenizer(valBatch['english_sentence'],padding = True, truncation = True, max_length = 256,return_tensors = "pt").input_ids.cuda()
                    tgt = tgttokenizer(valBatch['hindi_sentence'],padding = True, truncation = True, max_length = 256,return_tensors = "pt").input_ids.cuda()
                    logits = trf(src,tgt[:,:-1]).permute((0,2,1))
                    loss = CCE(logits,tgt[:,1:])
                    preds = logits.argmax(1)
                    total_corrects = (preds == tgt[:,1:]).sum()
                    accuracy = accuracy + (total_corrects/(logits.shape[-1]*logits.shape[-3]))
                    val_loss += loss.item()*src.size(0)
            metrics = {"step":step,"trainLoss":round((total_loss/(8000)),4),"valLoss":round((val_loss/575),4),"acc":round((accuracy.item()/5.75),4)}
            print(metrics)
            if (val_loss/575)< best_val_loss:
                best_val_loss = val_loss/575
                torch.save({'step': step,
                            'model_state_dict': trf.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': total_loss/8000,},"./Transformer.pt")
            total_loss = 0.0

  0%|          | 0/14295 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

{'step': 1000, 'trainLoss': 3.7074, 'valLoss': 2.9974, 'acc': 20.2888}


  0%|          | 0/72 [00:00<?, ?it/s]

{'step': 2000, 'trainLoss': 2.7834, 'valLoss': 2.5621, 'acc': 24.7936}


...

  0%|          | 0/72 [00:00<?, ?it/s]

{'step': 43000, 'trainLoss': 0.635, 'valLoss': 0.7249, 'acc': 61.5752}


  0%|          | 0/72 [00:00<?, ?it/s]

{'step': 44000, 'trainLoss': 0.5701, 'valLoss': 0.7107, 'acc': 63.14}


KeyboardInterrupt: 

In [18]:
trf = torch.load("Transformer.pt")

In [27]:
trf.keys()

dict_keys(['step', 'model_state_dict', 'optimizer_state_dict', 'loss'])